<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex14notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex14notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo14.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の FFT の関数
from scipy.fft import fft, ifft, fftfreq

# 科学技術計算のライブラリ SciPy の中の WAVE ファイルを扱うモジュール
from scipy.io.wavfile import read, write

---
## 演習問題 標本化
---



#### 準備

あとの実験で使う関数とデータを用意しておきます．

In [ ]:
# WAVE ファイルを読み込む関数
#
def readWAVE(filename):
    
    framerate, data = read(filename)

    # チャンネル数とフレーム数（データ点の数）を求める
    if data.ndim == 1:
        nchannels = 1
    else:
        nchannels = data.shape[1]
    nframes = data.shape[0]
    
    print('filename = ', filename)
    print('nchannels = ', nchannels)       # チャンネル数
    print('framerate = ', framerate)       # 標本化周波数
    print('nframes = ', nframes)             # フレーム数
    print('duration = {0:.2f}[sec]'.format(nframes / framerate))   # 長さ（秒単位）
    print('dtype = ', data.dtype)            # データ型（量子化ビット数に対応）

    assert data.dtype == 'uint8' or data.dtype == 'int16' or data.dtype == 'int32' or data.dtype == 'float32'
    
    # 最初の10秒分だけ取り出す（元がそれより短ければそのまま）
    nframesNew = min(framerate * 10, nframes)   
    if nchannels == 1:
        dataNew = data[:nframesNew]
    else:
        # 多チャンネル信号なら0番目と1番目の平均値を取り出す
        if data.dtype == 'float32':  # 浮動小数点数のときは [0, 1] の値なので普通に足して2で割る
            dataNew = (data[:nframesNew, 0] + data[:nframesNew, 1])/2
        else: # 整数型のときはオーバーフローする可能性があるので，いったん64bit整数にしてから
            data64 = (data[:nframesNew, 0].astype(np.int64) + data[:nframesNew, 1].astype(np.int64))//2
            dataNew = data64.astype(data.dtype)
    
    return framerate, dataNew

In [ ]:
# WAVE ファイルを入手
#
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar1-C.wav
fn = 'Sound-Guitar1-C.wav'

import os

if not os.path.exists(fn):
    print(f'{fn}のダウンロードがうまくできていないようです')

In [ ]:
# WAVE ファイルを読み込む
#
framerate, dat = readWAVE(fn)
dat = dat.astype(float)
dat -= np.mean(dat)
guitar = dat

In [ ]:
# WAVE ファイルを入手
#
### こちらのサイトの素材を利用させてもらってます http://www.ne.jp/asahi/music/myuu/wave/wave.htm

! wget -nc http://www.ne.jp/asahi/music/myuu/wave/cat1.wav
fnCat = 'cat1.wav'

import os

if not os.path.exists(fnCat):
    print(f'{fnCat}のダウンロードがうまくできていないようです')

#### 問題1

次のセルを実行すると，3つの正弦波のグラフが描かれます．
それぞれの周期[s]と周波数[Hz]を答えなさい．



In [ ]:
f = np.array([3*np.pi, 60*np.pi, 1200*np.pi])

fig, ax = plt.subplots(3, 1, figsize=(10, 10))
for i in range(3):

    t = np.linspace(0, 6*np.pi/f[i], num=1000)
    ax[i].plot(t, np.sin(f[i]*t))
    ax[i].axhline(0, color='gray')
    ax[i].set_xlabel('t [s]')

plt.show()

#### 問題2

次のセルを実行すると，`guitar` という配列に格納された信号という配列に格納された信号をグラフに描きます．
横軸は時間でその単位は[s]です．この信号は，音の波を等間隔に標本化したものです．
ここから標本化間隔[s]と標本化周波数[Hz]を求めなさい．



In [ ]:
N = len(guitar)
t = np.linspace(0, N/framerate, num=N, endpoint=False)
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(t, guitar, 'o')
ax.axhline(0, color='gray')
ax.set_xlim(0.2995, 0.3025)
ax.set_ylim(-150, 150)
ax.set_xlabel('t [s]')
plt.show()

#### 問題3

次のセルを実行すると，`guitar` に格納された信号に離散フーリエ変換を適用して振幅スペクトルを描きます．スペクトルの横軸は，フーリエ係数の番号 $k$ を表します．下段のスペクトルは，上段のもののの $1060 \leq k \leq 1090$ の範囲を拡大表示したものです．次の問に答えなさい．

(1) 次のセルを実行すると表示される `N` の値は，`guitar` に格納された値の数を表します．これと，問題2で求めた標本化周波数から，$k$番目のフーリエ係数が何[Hz]の波の成分に対応するかを求めなさい．

(2) 下段のスペクトルに示された周波数成分のうち，振幅の値が最も大きいものは，何[Hz]の音に相当するか答えなさい．




In [ ]:
print(f'N = {N}')
coef = fft(guitar) # DFT
amp = np.abs(coef)/N # 振幅スペクトル
fig, ax = plt.subplots(2, 1, figsize=(10, 10))
ax[0].stem(amp, markerfmt=' ', use_line_collection=True)
ax[0].set_xlim(0, 4000)
ax[0].set_xlabel('k')
ax[1].stem(amp, markerfmt=' ', use_line_collection=True)
ax[1].set_xlim(1060, 1090)
ax[1].set_xlabel('k')
plt.show()

#### 問題4

ウェブで検索する等して，次のものについて調べなさい．
- ヒトの可聴域（音として感じることのできる鼓膜振動の周波数範囲）
- CD-DA規格における標本化周波数（CD-DA(Compact Disc Digital Audio）は，音響信号を記録するCDのための規格）

後者は前者の最大値より高いはずだが，なぜそんなに高いのか理由を考えなさい．



###標本化定理の教えを守らないと...

標本化定理の教えを守らないとどういうことになるのか実験してみましょう．

In [ ]:
print('##### cat1 #####')
fr_cat1, cat1 = readWAVE('cat1.wav')
N_cat1 = len(cat1)
print()

`cat1` は前回も使ったもふもふの鳴き声のデータです．標本化周波数 44100[Hz] で標本化されています．次のセルを実行すると，その振幅スペクトルを描きます．

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
coef = fft(cat1)
amp = np.abs(coef)/N_cat1
freq = fftfreq(N_cat1, d=1.0/fr_cat1)
ax.stem(freq, amp, markerfmt=' ', use_line_collection=True, label='cat1')
ax.set_xlim(0, 6000)
ax.set_xlabel('frequency [Hz]')
ax.legend()
plt.show()

3000[Hz]以上の高い周波数の成分が含まれていますね．

この `cat1` の値を10個おきに取り出すことで，標本化間隔$10$倍，標本化周波数$\frac{1}{10}$で標本化しなおしてみます．新たな信号 `cat2` の標本化周波数は 4410 [Hz] となります．

In [ ]:
print('##### cat2 #####')
cat2 = cat1[::10] # cat1 を10個おきに標本化しなおす
N_cat2 = len(cat2)
fr_cat2 = fr_cat1//10
print(f'framerate = {fr_cat2}')
print(f'nframes = {N_cat2}')

fig, ax = plt.subplots(figsize=(10, 5))
t1 = np.linspace(0, N_cat1/fr_cat1, num=N_cat1, endpoint=False)
t2 = np.linspace(0, N_cat2/fr_cat2, num=N_cat2, endpoint=False)
ax.plot(t1, cat1, 'o', label='cat1')
ax.plot(t2, cat2, 'o', label='cat2')
ax.axhline(0, color='gray')
ax.set_xlim(0.5, 0.502)
ax.set_ylim(-16384, 16384)
ax.legend()
plt.show()

次のセルを実行すると，`cat1` と `cat2` をWAVE形式のファイルに保存し，それを自分の手元にダウンロードすることができます．聴き比べてみましょう．

In [ ]:
write('cat1.wav', fr_cat1, cat1.astype('int16'))
write('cat2.wav', fr_cat2, cat2.astype('int16'))
from google.colab import files
if 1 == 0: # ← の 0 を 1 に修正
    files.download('cat2.wav')
    files.download('cat1.wav')

`cat2.wav` の方はおかしなことになっていますね．振幅スペクトルを描いてみると...

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 10))
coef = [fft(cat1), fft(cat2)] # DFT
amp = [np.abs(coef[0])/N_cat1, np.abs(coef[1])/N_cat2]
freq = [fftfreq(N_cat1, d=1.0/fr_cat1), fftfreq(N_cat2, d=1.0/fr_cat2)]
for i in range(2):
    ax[i].stem(freq[i], amp[i], markerfmt=' ', use_line_collection=True, label=f'cat{i+1}')
    ax[i].set_xlim(0, 6000)
    ax[i].set_xlabel('frequency [Hz]')
    ax[i].legend()
plt.show()

`cat2` の標本化周波数は 4410 [Hz] です．標本化定理によれば，元の信号に含まれる成分の最高周波数がこの半分すなわち 2205 [Hz] 未満であれば，標本化した信号から元の成分を再現することができます．しかし，今の場合，元の信号 `cat1` には明らかに 2205 [Hz] 以上の成分が含まれていました．それを無理やり低い標本化周波数で標本化してしまいました．

`cat2` の振幅スペクトルを見ると，元の信号の 2205 [Hz] 以上の成分が， 2205 [Hz] を軸に折り返されて低い周波数の方へ現れてしまっています．このような現象を **エイリアシング** (aliasing) といいます．
`cat2` はエイリアシングのせいで元の音とは違う音に聞こえるようになってしまいました．